# Initialisation

In [1]:
root_dir = "/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/"

## General imports

In [2]:
import io
import csv
import os.path

import numpy as np
import numexpr as ne
import matplotlib.pyplot as plt

%matplotlib qt5

In [3]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
from posarmctools.ekfnavtools import *

In [5]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.PosarMCParameters import *

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
withPlots = 0

## EPSG transformations

In [8]:
import mpl_toolkits.basemap.pyproj as pyproj

In [9]:
# Define some common projections using EPSG codes
wgs84 = pyproj.Proj("+init=EPSG:4326") # LatLon with WGS84 datum used by GPS units and Google Earth
epsg3857 = pyproj.Proj("+init=EPSG:3857") # WGS84 / Pseudo Mercator
epsg3395 = pyproj.Proj("+init=EPSG:3395") # WGS84 / World Mercator
epsg3948 = pyproj.Proj("+init=EPSG:3948") # RGF93 / CC48 Projected coordinate system

epsg3xxx = epsg3948 # epsg3857 does not seem to keep distances

In [10]:
def transform( Long_Lat, shift=0, orig=(0,0) ):
    
    if shift:
        epsg = pyproj.transform(wgs84, epsg3xxx, Long_Lat[0], Long_Lat[1])
        epsg = (epsg[0] - orig[0], epsg[1] - orig[1])
    else:
        epsg = pyproj.transform(wgs84, epsg3xxx, Long_Lat[0], Long_Lat[1])
        
    return epsg

In [11]:
if epsg3xxx == epsg3857:
    origXY = (-180000, 6183000) # epsg 3857
    shiftXY = 1
    xlim = 17500
    ylim = 17500
    epsgStr = "epsg3857"
    
if epsg3xxx == epsg3948:
    origXY = (1360000, 7260000)
    shiftXY = 1
    xlim = 11000
    ylim = 12000
    epsgStr = "epsg3948"

## Scene remarkable points

In [12]:
def getLatLong( a, b ):
    Da = float(a.split("°")[0])
    r = a.split("°")[1]
    Ma = float(r.split("'")[0])
    r = r.split("'")[1]
    Sa = float(r.split("N")[0])
    
    Db = float(b.split("°")[0])
    r = b.split("°")[1]
    Mb = float(r.split("'")[0])
    r = r.split("'")[1]
    Sb = float(r.split("W")[0])
    
    return ( Da + Ma/60 + Sa/3600, -(Db + Mb/60 + Sb/3600) )

In [13]:
J1 = [ 48 + 28/60 + 8.08/3600, -(1 + 29/60 + 26.72/3600) ]
J2 = [ 48 + 32/60 + 31.59/3600, -(1 + 35/60 + 15.44/3600) ]
J3 = [ 48 + 32/60 + 36.80/3600, -(1 + 35/60 + 4.86/3600) ]   
J4 = [ 48 + 28/60 + 15.03/3600, -(1 + 29/60 + 16.23/3600) ]
J5 = getLatLong( "48°28'21.95N", "1°29'5.65W" )
J6 = getLatLong( "48°32'42.05N", "1°34'54.30W" )
J7 = getLatLong( "48°32'47.28N", "1°34'43.73W" )
J8 = getLatLong( "48°28'28.87N", "1°28'55.04W" )
J9 = getLatLong( "48°28'35.79N", "1°28'44.44W" )
J10 = getLatLong( "48°32'52.51N", "1°34'33.18W" )
J11 = [ 48 + 32 / 60 + 57.92 / 3600, -(1 + 34 / 60 + 22.25 / 3600) ]
J12 = [ 48 + 28 / 60 + 42.91 / 3600, -(1 + 28 / 60 + 33.18 / 3600) ]

J1_epsg = transform( J1[::-1], shiftXY, origXY )
J2_epsg = transform( J2[::-1], shiftXY, origXY )
J3_epsg = transform( J3[::-1], shiftXY, origXY )
J4_epsg = transform( J4[::-1], shiftXY, origXY )
J5_epsg = transform( J5[::-1], shiftXY, origXY )
J6_epsg = transform( J6[::-1], shiftXY, origXY )
J7_epsg = transform( J7[::-1], shiftXY, origXY )
J8_epsg = transform( J8[::-1], shiftXY, origXY )
J9_epsg = transform( J9[::-1], shiftXY, origXY )
J10_epsg = transform( J10[::-1], shiftXY, origXY )
J11_epsg = transform( J11[::-1], shiftXY, origXY )
J12_epsg = transform( J12[::-1], shiftXY, origXY )

np.save( root_dir + "J1_epsg", J1_epsg )
np.save( root_dir + "J2_epsg", J2_epsg )
np.save( root_dir + "J3_epsg", J3_epsg )
np.save( root_dir + "J4_epsg", J4_epsg )
np.save( root_dir + "J5_epsg", J5_epsg )
np.save( root_dir + "J6_epsg", J6_epsg )
np.save( root_dir + "J7_epsg", J7_epsg )
np.save( root_dir + "J8_epsg", J8_epsg )
np.save( root_dir + "J9_epsg", J9_epsg )
np.save( root_dir + "J10_epsg", J10_epsg )
np.save( root_dir + "J11_epsg", J11_epsg )
np.save( root_dir + "J12_epsg", J12_epsg )

# SBG data

## data path definitions

In [14]:
day = "2018_06_27"
prefix = "/home/pleroy/DATA/2018_06_27_LETG/dataLogger/session_0005/" + day + "/"

hours = ["09h00", "10h00", "11h00", "12h00"]
nbHours = len(hours)

logEkfEuler = [ prefix + h + "/sbgLogEkfEuler.dat" for h in hours ]
logEkfNav  = [ prefix + h + "/sbgLogEkfNav.dat" for h in hours ]
logUtcData = [ prefix + h + "/sbgLogUtcData.dat" for h in hours ]
logGpsPos = [ prefix + h + "/sbgLogGpsPos.csv" for h in hours ]
logGpsVel = [ prefix + h + "/sbgLogGpsVel.dat" for h in hours ]
logEventB = [ prefix + h + "/sbgLogEventB.dat" for h in hours ]

##  logGpsPos

In [15]:
idx_gpsPos_lat = 3
idx_gpsPos_long = 4
idx_gpsPos_alt = 5
idx_gps_lat = 1
idx_gps_long = 2
idx_gps_alt = 3

def loadLogGpsPos( gps, n ):
    for idx in range(n):
        gps.append( np.loadtxt( logGpsPos[idx], skiprows = 1, delimiter = ',',
                               usecols = (0, idx_gpsPos_lat, idx_gpsPos_long, idx_gpsPos_alt) ) )

### Load data

In [16]:
gps=[]
loadLogGpsPos( gps, nbHours )

In [17]:
gps_all = gps[0]
for idx in range(1, 4):
    gps_all = np.concatenate( (gps_all, gps[idx]), axis=0 )

In [18]:
Lat = gps_all[:,idx_gps_lat]
Long = gps_all[:,idx_gps_long]
Alt = gps_all[:,idx_gps_alt]

### Plot navigation data jointly with track references

In [19]:
plt.figure()
ax = plt.subplot(111)
plotLongLatAndTrackReferences( ax, Long, Lat )
plotRunaway( ax )
plt.title( "GPS data and track references\n" + prefix )
plt.grid()

### Shift timestamps

In [20]:
gpsTimestamps = gps_all[:, 0]

In [21]:
for k in range(1,gpsTimestamps.shape[0]):
    if gpsTimestamps[k] < gpsTimestamps[k-1]:
        print("k = {}".format(k))

k = 3734
k = 25209
k = 46684


In [22]:
k1 = 3734
k2 = 25209
k3 = 46684
gpsNewTimestamps = np.concatenate( (gpsTimestamps[:k1],
                                    gpsTimestamps[k1:k2] + 1*2**32, 
                                    gpsTimestamps[k2:k3] + 2*2**32, 
                                    gpsTimestamps[k3:] + 3*2**32) )

In [23]:
if withPlots:
    plt.figure()
    plt.plot( gpsNewTimestamps, 'o' )
    plt.plot( gpsTimestamps,'.' )
    plt.grid()

### Transform GPS data

In [24]:
idx_lat = 1
idx_long = 2

In [25]:
idx = 0
x0, y0 = transform( (gps[idx][:,idx_long], gps[idx][:,idx_lat]), shiftXY, origXY )

idx = 1
x1, y1 = transform( (gps[idx][:,idx_long], gps[idx][:,idx_lat]), shiftXY, origXY )

idx = 2
x2, y2 = transform( (gps[idx][:,idx_long], gps[idx][:,idx_lat]), shiftXY, origXY )

idx = 3
x3, y3 = transform( (gps[idx][:,idx_long], gps[idx][:,idx_lat]), shiftXY, origXY )

x = [x0, x1, x2, x3]
y = [y0, y1, y2, y3]

In [26]:
plt.figure()
for idx in range(4):
    plt.plot( x[idx], y[idx], label=hours[idx] )
    
plt.plot( J1_epsg[0], J1_epsg[1], 'oy', markeredgecolor = 'black' )
plt.plot( J2_epsg[0], J2_epsg[1], 'oy', markeredgecolor = 'black' )
plt.plot( J11_epsg[0], J11_epsg[1], 'oy', markeredgecolor = 'black' )
plt.plot( J12_epsg[0], J12_epsg[1], 'oy', markeredgecolor = 'black' )

plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend()
title = epsgStr + day
plt.title(title)
plt.axes().set_aspect('equal')

In [27]:
gps_proj = transform( (Long, Lat), shiftXY, origXY )
np.save( root_dir + "gps_epsg_transform", gps_proj )

## logGpsVel

### Load data

In [28]:
idx_gpsVel_north = 3
idx_gpsVel_east = 4
idx_gpsVel_down = 5
idx_gpsVel_course = 9

vel=[]
for idx in range(4):
    vel.append( np.loadtxt( logGpsVel[idx], skiprows = 1,
                           usecols = (0, idx_gpsVel_north, idx_gpsVel_east, idx_gpsVel_down, idx_gpsVel_course) ) )

In [29]:
vel_all = vel[0]
for idx in range(1, 4):
    vel_all = np.concatenate( (vel_all, vel[idx]), axis=0 )

In [30]:
Vel = ( vel_all[:,1]**2 + vel_all[:,2]**2 + vel_all[:,3]**2) **0.5
course = vel_all[:,4]

## sbgLogEventB

### Load data

In [31]:
event = []
for idx in range(4):
    event.append( np.loadtxt( logEventB[idx], skiprows = 1, usecols = 0 ) )

indexes = []
indexes.append( np.arange(event[0].size) )
for idx in range(1, 4):
    indexes.append( np.arange(event[idx].size)+indexes[idx-1][-1]+1)

In [32]:
event_all = event[0]
for idx in range(1, 4):
    event_all = np.concatenate( (event_all, event[idx]), axis=0 )

In [33]:
plt.figure()
for idx in range(4):
    plt.plot( indexes[idx], event[idx]/1e6, ".", label = "{}".format(hours[idx]) )
    
plt.legend()
plt.grid()

In [34]:
plt.figure()
for idx in range(4):
    plt.plot( indexes[idx][:-1], np.diff(event[idx])/1e6, ".", label = "{}".format(hours[idx]) )
    
plt.legend()
plt.grid()

## logUtcData

### Load data

In [35]:
#timeStamp status year month day hour minute second nanoSecond gpsTimeOfWeek
idx_h = 5
idx_m = 6
idx_s = 7
idx_nano = 8

utc = []
for idx in range(4):
    utc.append( np.loadtxt( logUtcData[idx], skiprows = 1, usecols = (0, idx_h, idx_m, idx_s, idx_nano) ) )

In [36]:
utc_all = utc[0]
for idx in range(1, 4):
    utc_all = np.concatenate( (utc_all, utc[idx]), axis=0 )

In [37]:
printUtc( 0, utc_all )
printUtc( -1, utc_all )

9.0 : 13.0 : 50.750
12.0 : 45.0 : 37.775


In [38]:
utcTimestamps = utc_all[:, 0]

### Shift timestamps

In [39]:
for k in range(1,utcTimestamps.shape[0]):
    if utcTimestamps[k] < utcTimestamps[k-1]:
        print("k = {}".format(k))

k = 149353
k = 1008346
k = 1867340


In [40]:
k1 = 149353
k2 = 1008346
k3 = 1867340
utcNewTimestamps = np.concatenate( (utcTimestamps[:k1], 
                                    utcTimestamps[k1:k2] + 1*2**32,
                                    utcTimestamps[k2:k3] + 2*2**32, 
                                    utcTimestamps[k3:] + 3*2**32) )

In [41]:
if withPlots:
    plt.figure()
    plt.plot( utcNewTimestamps, 'o' )
    plt.plot( utcTimestamps,'.' )
    plt.grid()

In [42]:
printUtc( 0, utc_all )
printUtc( k1, utc_all )
printUtc( k2, utc_all )
printUtc( k3, utc_all )
printUtc( -1, utc_all )

9.0 : 13.0 : 50.750
9.0 : 26.0 : 17.515
10.0 : 37.0 : 52.480
11.0 : 49.0 : 27.450
12.0 : 45.0 : 37.775


## logEkfEuler

### Load data

In [43]:
idx_timeStamp = 0
idx_roll = 1
idx_pitch = 2
idx_yaw = 3
idx_rollStdDev = 4
idx_pitchStdDev = 5
idx_yawStdDev = 6
idx_status = 7

In [44]:
euler = []
for idx in range(4):
    euler.append( np.loadtxt( logEkfEuler[idx], skiprows = 1,
                             usecols = (0, idx_roll, idx_pitch, idx_yaw) ) )

In [45]:
euler_all = euler[0]
for idx in range(1, 4):
    euler_all = np.concatenate( (euler_all, euler[idx]), axis=0 )

In [46]:
eulerTimestamps = euler_all[:,0]
roll = euler_all[:,1]
pitch = euler_all[:,2]
yaw = euler_all[:,3]

### Shift timestamps

In [47]:
for k in range(1,eulerTimestamps.shape[0]):
    if eulerTimestamps[k] < eulerTimestamps[k-1]:
        print("k = {}".format(k))

k = 149353
k = 1008346
k = 1867340


In [48]:
k1 = 149353
k2 = 1008346
k3 = 1867340
eulerNewTimestamps = np.concatenate( (eulerTimestamps[:k1], 
                                      eulerTimestamps[k1:k2] + 1*2**32,
                                      eulerTimestamps[k2:k3] + 2*2**32, 
                                      eulerTimestamps[k3:] + 3*2**32) )

## logEkfNav

### Load data

In [49]:
idx_timeStamp = 0
idx_velNorth = 1
idx_velEast = 2
idx_velDown = 3
idx_velNorth_StdDev = 4
idx_velEast_StdDev = 5
idx_velDown_StdDev = 6
idx_ekf_Lat = 7
idx_ekf_Long = 8
idx_ekf_Alt = 9
idx_ekf_undulation = 10
dx_ekf_Lat_StdDev = 11
idx_ekf_Long_StdDev = 12
idx_ekf_Alt_StdDev = 13
idx_status = 14

In [50]:
nav = []
for idx in range(4):
    nav.append( np.loadtxt( logEkfNav[idx], skiprows = 1,
                             usecols = (0, idx_ekf_Lat, idx_ekf_Long, idx_ekf_Alt) ) )

In [51]:
nav_all = nav[0]
for idx in range(1, 4):
    nav_all = np.concatenate( (nav_all, nav[idx]), axis=0 )

In [52]:
navTimestamps = nav_all[:,0]
nav_Lat = nav_all[:,1]
nav_Long = nav_all[:,2]
nav_Alt = nav_all[:,3]

### Shift timestamps

In [53]:
for k in range(1,navTimestamps.shape[0]):
    if navTimestamps[k] < navTimestamps[k-1]:
        print("k = {}".format(k))

k = 149353
k = 1008346
k = 1867340


In [54]:
k1 = 149353
k2 = 1008346
k3 = 1867340
navNewTimestamps = np.concatenate( (navTimestamps[:k1], 
                                    navTimestamps[k1:k2] + 1*2**32,
                                    navTimestamps[k2:k3] + 2*2**32, 
                                    navTimestamps[k3:] + 3*2**32) )

In [55]:
nav_proj = transform( (nav_Long, nav_Lat), shiftXY, origXY )
np.save( root_dir + "nav_epsg_transform", nav_proj )

In [56]:
plt.figure()
plt.plot( nav_proj[0], nav_proj[1], label = "nav" )
plt.plot( gps_proj[0], gps_proj[1], label = "gps" )
plt.plot(J1_epsg[0], J1_epsg[1], "Dy", markerEdgecolor="k")
plt.plot(J2_epsg[0], J2_epsg[1], "Dy", markerEdgecolor="k")
plt.plot(J3_epsg[0], J3_epsg[1], "Dy", markerEdgecolor="k")
plt.plot(J4_epsg[0], J4_epsg[1], "Dy", markerEdgecolor="k")
plt.legend()
plt.grid()
plt.axes().set_aspect('equal')

In [57]:
plt.figure()
plt.plot(navNewTimestamps, nav_Alt, label = "nav")
plt.plot(gpsNewTimestamps, Alt, '.', label = "gps")
plt.legend()
plt.grid()

# PoSAR-MC data

## Parameters

In [58]:
dates = ["12_39_39", "12_45_51", "12_52_49", "12_59_03",
         "13_05_40", "13_11_42", "13_18_13", "13_24_11",
         "13_30_53", "13_36_46", "13_43_13", "13_49_07"]

data_date = [ "2018_06_27_" + d for d in dates ]
data_dir = [root_dir +  d for d in data_date]

firstRecord = 0
lastRecord = [682, 780, 656, 736,
              640, 750, 648, 744,
              656, 742, 646, 756
             ]

## Process timeStamps.data

In [59]:
def checkTimestamps(idx, dropFirst=0, withPlot=1):
    
    #====================================================================
    # the last value of the file may be erroneous, this should be checked
    #====================================================================

    #===================================================================
    # the first value may be erroneous, one may have to shift all values
    #===================================================================
    
    timeStampsFile = data_dir[idx] + "/" + data_date[idx] + "_timeStamps.data"
    print(timeStampsFile)
    bufferNumber, timeStamp = np.loadtxt( timeStampsFile, skiprows = 1, unpack=True )

    if dropFirst:
        bufferNumber = bufferNumber[:-1]
        timeStamp = timeStamp[1:]
    
    if withPlot:
        plt.figure()
        plt.subplot(211)
        plt.plot( timeStamp, ".", label=data_date[idx] )
        plt.grid()
        plt.legend()
        plt.title("timeStamp wrt fileNumber / " + data_date[idx] )
        plt.subplot(212)
        plt.plot( np.diff(timeStamp)/1e6, "." )
        plt.grid()
        plt.title( "diff(timeStamp) wrt fileNumber / " + data_date[idx] )
    
    return bufferNumber, timeStamp

In [61]:
dropFirst=1
withPlot=0
bufferNumber0, timeStamp0 = checkTimestamps(0, dropFirst, withPlot)

/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_39_39/2018_06_27_12_39_39_timeStamps.data


In [62]:
dropFirst=0
withPlot=0
bufferNumber1, timeStamp1 = checkTimestamps(1, dropFirst, withPlot)

/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_45_51/2018_06_27_12_45_51_timeStamps.data


In [63]:
dropFirst=0
withPlot=0
bufferNumber2, timeStamp2 = checkTimestamps(2, dropFirst, withPlot)

/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/2018_06_27_12_52_49/2018_06_27_12_52_49_timeStamps.data


In [64]:
dropFirst=1
withPlot=0
bufferNumber4, timeStamp4 = checkTimestamps(4, dropFirst, withPlot)

/home/pleroy/DATA/2018_06_27_LETG/2018_06_27/jde/2018_06_27_13_05_40/2018_06_27_13_05_40_timeStamps.data


**Be carefull here, there could be a jump in the timeStamp values due to the counter saturation at 2^32. The mitigation is following.**

In [65]:
timestampsShift0 = 2 # this is for the SBG internal us loopback
timestampsShift1 = 2 # this is for the SBG internal us loopback
timestampsShift2 = 2 # this is for the SBG internal us loopback
timestampsShift4 = 2 # this is for the SBG internal us loopback

In [66]:
newTimeStamp0 = timeStamp0 + timestampsShift0 * 2**32
newTimeStamp1 = timeStamp1 + timestampsShift1 * 2**32
newTimeStamp2 = timeStamp2 + timestampsShift2 * 2**32
newTimeStamp4 = timeStamp4 + timestampsShift4 * 2**32

In [67]:
printUtc( np.where(utcNewTimestamps < newTimeStamp0[0])[0][-1], utc_all )
printUtc( np.where(utcNewTimestamps < newTimeStamp1[0])[0][-1], utc_all )
printUtc( np.where(utcNewTimestamps < newTimeStamp2[0])[0][-1], utc_all )
printUtc( np.where(utcNewTimestamps < newTimeStamp4[0])[0][-1], utc_all )

10.0 : 39.0 : 38.365
10.0 : 45.0 : 50.465
10.0 : 52.0 : 48.425
11.0 : 5.0 : 38.865


## Selected record

In [68]:
selectedRecord = 4

### Read parameters

In [69]:
params_filename = data_dir[selectedRecord] + "/" + data_date[selectedRecord] + "_parameters.xml"
params = PosarMCParameters_v2( params_filename )

In [70]:
def getNbRecords( first, last, params ):
    nbRecords = [int((l - first) / params.buffersPerFile + 1) for l in last]
    return nbRecords

In [71]:
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
rampsPerBuffer = params.rampsPerBuffer
samplesPerFile = params.samplesPerRamp * params.rampsPerFile
buffersPerFile = params.buffersPerFile
T_files = samplesPerFile / 10e6

nbRecords = getNbRecords(firstRecord, lastRecord, params)

print( "nbRecords = {}".format(nbRecords) )

nbRecords = [342, 391, 329, 369, 321, 376, 325, 373, 329, 372, 324, 379]


### Check the existence of the files

In [72]:
fileList = []

for k in range(firstRecord, lastRecord[selectedRecord], buffersPerFile):
    filename = data_dir[selectedRecord] + "/record" + str(k) + ".bin"
    ret = os.path.isfile(filename)
    if ret == False:
        fileList.append(0)
    else:
        fileList.append(1)

idxFileList = np.arange( len( fileList ) ) * buffersPerFile + firstRecord

In [73]:
plt.figure()
plt.plot(idxFileList, fileList, 'o-')
plt.grid()

## Data interpolation

In [74]:
newTimeStamp = { 0:newTimeStamp0, 
                1:newTimeStamp1, 
                2:newTimeStamp2,
                4:newTimeStamp4}
selectedTimeStamp = newTimeStamp[selectedRecord]

bufferNumber = { 0:bufferNumber0,
               1:bufferNumber1,
               2:bufferNumber2,
               4:bufferNumber4}
selectedBufferNumber = bufferNumber[selectedRecord]

### logEkfEuler

In [75]:
# interpolate euler data for all timeStamps
roll_selection = np.interp( selectedTimeStamp, eulerNewTimestamps, roll )
pitch_selection = np.interp( selectedTimeStamp, eulerNewTimestamps, pitch )
yaw_selection = np.interp( selectedTimeStamp, eulerNewTimestamps, yaw )

In [76]:
if withPlots:
    plt.figure()
    ax = plt.subplot(211)
    plt.plot(eulerNewTimestamps, yaw, label="yaw")
    plt.legend()
    plt.grid()
    plt.subplot(212, sharex=ax)
    plt.plot(gpsNewTimestamps, Alt, label="alt")
    plt.legend()
    plt.grid()

### logGpsPos

In [77]:
velNewTimestamps = gpsNewTimestamps

def getInterpolatedGps( timeStamp ):
    
    Lat_records = np.interp( timeStamp, gpsNewTimestamps, Lat )
    Long_records = np.interp( timeStamp, gpsNewTimestamps, Long )
    Alt_records = np.interp( timeStamp, gpsNewTimestamps, Alt )
    Vel_records = np.interp( timeStamp, velNewTimestamps, Vel )
    course_records = np.interp( timeStamp, velNewTimestamps, course )
    
    return Lat_records, Long_records, Alt_records, Vel_records, course_records

In [78]:
Lat_Long_Alt_Vel_course = getInterpolatedGps( selectedTimeStamp )

In [79]:
epsg = transform( (Lat_Long_Alt_Vel_course[1], Lat_Long_Alt_Vel_course[0]), shiftXY, origXY )

In [80]:
def addRemarkablePoints():
    plt.plot( J1_epsg[0], J1_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J2_epsg[0], J2_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J3_epsg[0], J3_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J4_epsg[0], J4_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J5_epsg[0], J5_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J6_epsg[0], J6_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J7_epsg[0], J7_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J8_epsg[0], J8_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J9_epsg[0], J9_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J10_epsg[0], J10_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J11_epsg[0], J11_epsg[1], 'oy', markeredgecolor = 'black' )
    plt.plot( J12_epsg[0], J12_epsg[1], 'oy', markeredgecolor = 'black' )

In [81]:
plt.figure()
for idx in range(4):
    plt.plot( x[idx], y[idx], label=hours[idx] )
    
addRemarkablePoints()
    
plt.plot( epsg[0], epsg[1], ".r", markeredgecolor='black', label=data_date[selectedRecord] )

plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend()
title = epsgStr + " " + day
plt.title(title)
plt.axes().set_aspect('equal')
ax = plt.gca()
ax.set_xlim([0, xlim])
ax.set_ylim([0, ylim])
plt.savefig( data_dir[selectedRecord] + "/" + title + ".png", bbox_inches='tight')

## Plot navigation data and record periods

In [82]:
plt.figure()

ax = plt.subplot(111)

plt.plot( Lat_Long_Alt_Vel_course[1], Lat_Long_Alt_Vel_course[0], ".r", markeredgecolor='black', 
         label=data_date[selectedRecord] )

title = "record locations-" + data_date[selectedRecord]
plt.title(title)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.legend()
plt.grid()

plt.savefig( data_dir[selectedRecord] + "/" + title + ".png", bbox_inches='tight')

In [83]:
plt.figure()
plt.plot( gpsNewTimestamps, Alt )
plt.plot( selectedTimeStamp, Lat_Long_Alt_Vel_course[2], "yD", markeredgecolor = 'black' )
plt.grid()
title = "Altitude - " + data_date[selectedRecord] + " - files {} to {}".format(firstRecord, lastRecord[selectedRecord])
plt.title( title )
plt.savefig( data_dir[selectedRecord] + "/" + title + ".png", bbox_inches='tight')

In [84]:
plt.figure()
for idx in range(4):
    plt.plot( x[idx], y[idx], label=hours[idx] )

addRemarkablePoints()

plt.plot( epsg[0], epsg[1], ".r", markeredgecolor='black', label=data_date[selectedRecord] )

plt.xlabel("x")
plt.ylabel("y")
plt.grid()
plt.legend()
title = "epsg3857 " + day
plt.title(title)
plt.axes().set_aspect('equal')
ax = plt.gca()
ax.set_xlim([0, xlim])
ax.set_ylim([0, ylim])
plt.savefig( data_dir[selectedRecord] + "/" + title + ".png", bbox_inches='tight')

# Track approximation

In [85]:
files = "files {} to {}".format( firstRecord, nbRecords[selectedRecord] )

In [86]:
plt.figure()

title = "altitude - velocity - course " + data_date[selectedRecord]

plt.subplot(311)
plt.title( title )
plt.plot( selectedTimeStamp, Lat_Long_Alt_Vel_course[2], 'Dy', markeredgecolor = 'k', label="altitude " + files )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( selectedTimeStamp, Lat_Long_Alt_Vel_course[3], 'Dy', markeredgecolor = 'k', label="velocity " + files )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( selectedTimeStamp, Lat_Long_Alt_Vel_course[4], 'Dy', markeredgecolor = 'k', label="course " + files )
plt.legend()
plt.grid()

plt.savefig( data_dir[selectedRecord] + "/" + title + ".png", bbox_inches='tight')

In [87]:
plt.figure()

title = "pitch - roll - yaw " + data_date[selectedRecord]

plt.subplot(311)
plt.title( title )
plt.plot( selectedTimeStamp, pitch_selection * 180 / np.pi, 'Dy', markeredgecolor = 'k', label="pitch [°] " + files )
plt.legend()
plt.grid()

plt.subplot(312)
plt.plot( selectedTimeStamp, roll_selection * 180 / np.pi, 'Dy', markeredgecolor = 'k',  label="roll [°] " + files )
plt.legend()
plt.grid()

plt.subplot(313)
plt.plot( selectedTimeStamp, yaw_selection * 180 / np.pi, 'Dy', markeredgecolor = 'k',  label="yaw [°] " + files )
plt.legend()
plt.grid()

plt.savefig( data_dir[selectedRecord] + "/" + title + ".png", bbox_inches='tight')

## Orientation vector

In [88]:
# East-West J1 => J2    12_39_39
# East-West J4 => J3    12_52_49
# East-West J5 => J6    13_05_40
# East-West J8 => J7    13_18_13
# East-West J9 => J10   13_30_53
# East-West J12 => J11  13_43_13
xA = J5_epsg[0]
yA = J5_epsg[1]
xB = J6_epsg[0]
yB = J6_epsg[1]

In [89]:
xAB = (xB - xA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5
yAB = (yB - yA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5

In [90]:
# unit vectors
ux = ( xAB, yAB )
uy = ( -yAB, xAB )

In [92]:
import json

In [93]:
track_model = { "trackModel" : "model(x, p) = p[1]*x + p[2]",
               "ux" : (ux[0], ux[1]),
               "uy" : (uy[0], uy[1]),
               "origX" : origXY[0],
               "origY" : origXY[1]
              }

In [94]:
trackFilename = data_dir[selectedRecord] + "/track_model.json"
with open( trackFilename, 'w' ) as f:
    json.dump( track_model, f )

In [96]:
plt.figure()
plt.plot(gps_proj[0], gps_proj[1])
plt.grid()
ax = plt.gca()
ax.set_aspect("equal")
title = "transformation of WGS84 to " + epsgStr
plt.title(title)

In [97]:
x_selection = np.interp( selectedTimeStamp, gpsNewTimestamps, gps_proj[0] )
y_selection = np.interp( selectedTimeStamp, gpsNewTimestamps, gps_proj[1] )
np.save( data_dir[selectedRecord] + "/track_selection_proj", np.stack( (x_selection, y_selection ), -1 ) )

In [98]:
plt.figure()

plt.plot( gps_proj[0], gps_proj[1], "gray" )

plt.plot( x_selection, y_selection, 'Dy', markerEdgecolor='k',
        label = "files {} to {} ".format( firstRecord, nbRecords[selectedRecord] ) + data_date[selectedRecord] )

ax = plt.gca()
ax.set_aspect("equal")
plt.xlabel("location of the records in the scene coordinate system\n")
plt.grid()
plt.legend()
ax = plt.gca()
ax.set_xlim([0, xlim])
ax.set_ylim([0, ylim])

(0, 12000)

# Interpolate navigation data for all ramps

## logGpsPos for rampDown
Two options: numpy.interp vs scipy.interpolate.interp1d (with kind='linear')

In [140]:
# rampNumber timeStamp x y z
nbRecordToAdd = 10

rampNumber = np.arange( rampsPerFile * (nbRecords[selectedRecord] + nbRecordToAdd) )


rampNumber_ext = np.arange(nbRecords[selectedRecord] + nbRecordToAdd) * rampsPerFile

lastDelta = selectedTimeStamp[-1] - selectedTimeStamp[-2]
# there is a difference between the number of valid timestamps and the number of records
diffValidTimestampsNbRecords = selectedTimeStamp.size - nbRecords[selectedRecord]
addedTimestamps = np.arange( 1, nbRecordToAdd - diffValidTimestampsNbRecords + 1 ) * lastDelta
print("lastDelta = {}".format(lastDelta) )
timeStamp_ext = np.concatenate( (selectedTimeStamp, 
                                 selectedTimeStamp[-1] + addedTimestamps ) )

lastDelta = 900002.0


### xyz

In [143]:
timeStamp = np.interp( rampNumber, rampNumber_ext, timeStamp_ext)
gps_x = np.interp( timeStamp, gpsNewTimestamps, gps_proj[0] )
gps_y = np.interp( timeStamp, gpsNewTimestamps, gps_proj[1] )
gps_z = np.interp( timeStamp, gpsNewTimestamps, Alt )
xyz_proj_allRamps = np.stack( (rampNumber, timeStamp, gps_x, gps_y, gps_z), -1 )

In [144]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_gps", xyz_proj_allRamps )

In [145]:
plt.figure()
plt.plot( x_selection, y_selection, 
         'Dy', markerEdgecolor='k', label = "files {} to {}".format( firstRecord, nbRecords[selectedRecord] ) )
plt.plot( gps_x, gps_y, '.b', label = 'ramps' )
plt.legend()
plt.title("xy")
plt.grid()

### lla latitude longitude altitude

In [146]:
timeStamp = np.interp( rampNumber, rampNumber_ext, timeStamp_ext)
allRamps_lat = np.interp( timeStamp, gpsNewTimestamps, Lat )
allRamps_long = np.interp( timeStamp, gpsNewTimestamps, Long )
allRamps_alt = np.interp( timeStamp, gpsNewTimestamps, Alt )
latLongAlt_allRamps = np.stack( (rampNumber, timeStamp, allRamps_lat, allRamps_long, allRamps_alt), -1 )

In [147]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_latLongAlt_gps", latLongAlt_allRamps )

## logEkfNav rampDown

In [ ]:
nav_x = np.interp( timeStamp, navNewTimestamps, nav_proj[0] )
nav_y = np.interp( timeStamp, navNewTimestamps, nav_proj[1] )
nav_z = np.interp( timeStamp, navNewTimestamps, nav_Alt )
nav_xyz_allRamps = np.stack( (rampNumber, timeStamp, 
                              nav_x, nav_y, nav_z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_nav", nav_xyz_allRamps )

In [148]:
nav_allRamps_lat = np.interp( timeStamp, navNewTimestamps, nav_Lat )
nav_allRamps_long = np.interp( timeStamp, navNewTimestamps, nav_Long )
nav_allRamps_alt = np.interp( timeStamp, navNewTimestamps, nav_Alt )
nav_latLongAlt_allRamps = np.stack( (rampNumber, timeStamp, 
                              nav_allRamps_lat, nav_allRamps_long, nav_allRamps_alt), -1 )

In [149]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_latLongAlt_nav", nav_latLongAlt_allRamps )

## Compare gps and nav

In [ ]:
plt.figure()
plt.plot( gps_x, gps_y, label="gps" )
plt.plot( nav_x, nav_y, label="nav" )
plt.legend()
plt.grid()
plt.title("x-y")

In [ ]:
plt.figure()
plt.plot( gps_z, label="gps"  )
plt.plot( nav_z, label="nav" )
plt.legend()
plt.grid()
plt.title("z")

## logGpsPos rampUp

In [ ]:
# rampNumber timeStamp x y z

timeStamp_ext_up = np.concatenate( 
    (selectedTimeStamp, [ selectedTimeStamp[-1] + selectedTimeStamp[-1] - selectedTimeStamp[-2] ] ) ) \
    + params.configuredTRamp / 2

timeStamp = np.interp( rampNumber, rampNumber_ext, timeStamp_ext_up)
x = np.interp( timeStamp, gpsNewTimestamps, gps_proj[0] )
y = np.interp( timeStamp, gpsNewTimestamps, gps_proj[1] )
z = np.interp( timeStamp, gpsNewTimestamps, Alt )
xyz_proj_allRamps = np.stack( (rampNumber, timeStamp, x, y, z), -1 )

In [ ]:
np.save( data_dir[selectedRecord] + "/rampNumber_timeStamp_xyz_rampUp", xyz_proj_allRamps )

In [ ]:
plt.figure()
plt.plot(rampNumber_ext, timeStamp_ext, 'o')
plt.plot(rampNumber_ext, timeStamp_ext_up, '.')